In [1]:
!pip3 -q install --upgrade pip
!pip3 -q install python-dotenv

In [30]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
!pip3 -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding ipywidgets

In [22]:
!conda update pytorch torchvision -c pytorch


PackageNotInstalledError: Package is not installed in prefix.
  prefix: /Users/alwaleedalmajlad/opt/anaconda3
  package name: pytorch




In [31]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [63]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

/Users/alwaleedalmajlad/Desktop/training/Projects/QualityChatBot/chatenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [5]:
loader = DirectoryLoader('files/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [6]:
len(documents)

251

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

671

In [10]:
texts[3]

Document(page_content='Page 3 of 251 \n \nID Chapter \nCode  Chapter Name  Number \nof \nStandards  Number \nof Sub -\nStandards  \n1 LD Effective Leadership  7 50 \n2 PCC Person -Centered Care  9 45 \n3 CP Clinical Privileging  8 37 \n4 SSL Safe Staffing Levels  7 108 \n5 ED Emergency Department  11 50 \n6 SR Stroke -Readiness  7 33 \n7 TR Trauma -Readiness  5 32 \n8 ACS ACS-Readiness  5 24 \n9 VTE Venous Thromboembolism  2 8 \n10 POR  Perioperative Safety  7 39 \n11 SLB Safe labor and birth  10 47 \n12 RRS Rapid Response System  4 26 \n13 PT Patient Transfer  7 35 \n14 ICC Adult Critical Care Services  9 40 \n15 NIC Neonatal Care Safety  7 56 \n16 EOL End-of-Life Care  7 26 \n17 BTS Blood Transfusion Safety  11 43 \n18 MM  Medication Safety  10 64 \n19 RS Radiation Safety  7 32 \n20 CSS Central Sterile Service Department  10 57 \n21 HAI Healthcare Associated Infection  6 43 \n22 FS Fire Safety  11 54 \n23 PHE Physical Environment  9 51 \n24 HOU  Housing  4 19 \n    Total:  180 1019',

In [48]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", 
                                                      model_kwargs={"device": "cpu"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [49]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

In [51]:
from pathlib import Path

# Check if the vector database already exists
db_path = Path('db')
if db_path.exists():
    vectordb = Chroma(persist_directory='db', embedding_function=embedding)
else:
    vectordb = Chroma.from_documents(documents=texts, embedding=embedding, persist_directory='db')

In [52]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [53]:
# # Now we can load the persisted database from disk, and use it as normal. 
# vectordb = Chroma(persist_directory=persist_directory, 
#                   embedding_function=embedding)

In [54]:
retriever = vectordb.as_retriever()

In [17]:
docs = retriever.get_relevant_documents("Mention 5 standards for hipsotial leadership?")

In [33]:
len(docs)

4

In [34]:
docs[0]

Document(page_content='to be the healthiest they can be.   \nLeadership and governa nce are core component of a resilient healthcare system, that plays an intrinsic \nrole within every layer of a healthcare organization. Leadership sets direction and oversight at each \nlevel within an organization while governance ensures that accountability  and organization outcome s \nare achieved . The standards were designed to reflect the international shift to healthcare models such \nas, integrated care models, accountable Care Organizations, value -based healthcare and Bundled \nPayments.  \nLeadership and governance can directly and indirectly play a role in  healthcare outcomes and \nchallenges  the delivery of quality healthcare services in an equitable manner. Inadequate healthcare \nleadership and/or governance can affect patient safety, patien t and employee satisfaction, efficiency \nand effectiveness, and healthcare costs.', metadata={'page': 3, 'source': 'files/CBAHI 4TH EDITION Nati

In [66]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [67]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [68]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(f"page number {source.metadata['page']}")

In [69]:
query = """
explain for me the hospital has systems to support the active involvement of patients and families in 
communication
"""
llm_response = qa_chain(query)
process_llm_response(llm_response)

The hospital has implemented structures and processes to support the active involvement of patients and
families in communication. This includes easy access to care and services for patients and families, timely
and accurate access to patient information between healthcare providers, and multiple mechanisms for patients
and families to access their own information. The hospital encourages the use of solutions that exchange
information across providers and patients, allowing for the linking of healthcare data across the continuum of
care. Additionally, the hospital is responsive to patient and family feedback, including concerns, queries,
and preferences. Overall, these systems and processes aim to enhance person-centered care and promote
effective communication between healthcare providers, patients, and their families.


Sources:
page number 13
page number 13


In [28]:
query = "We need to asses department based on The hospital supports staff well-being. Can you give me the point we should evaluted on it ??"
llm_response = qa_chain(query)
process_llm_response(llm_response)

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [ ]:
query = "whats is the radiology department standards ??"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The radiology department standards include having a system to identify patient radiation doses, setting
investigation triggers for common radiological examinations, having a radiation quality assurance program,
ensuring reliable and calibrated equipment, conducting periodic checks for radiation sources, and having
emergency plans and safety programs in place for non-ionizing radiation. These standards are outlined in the
CBAHI Radiation Safety Priority document and are based on international recommendations and guidelines.


Sources:
page number 222
page number 223
page number 220


In [59]:
query = "explain for me the calibrated equipment standards ??"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 I do not see any mention of calibrated equipment standards in the given context. It is possible that the
standards for calibrated equipment are not specifically mentioned in this document.


Sources:
page number 228
page number 227
page number 229
